# Analysing NN dynamics using HSIC Criterion - Intermediate layers

In [ ]:
# importing PyGlow modules
import glow
from glow.layers import Dense, Dropout, Conv2d, Flatten, HSICoutput
from glow.datasets import mnist, cifar10
from glow.models import IBSequential, Sequential, HSICSigma
from glow.information_bottleneck.estimator import HSIC

In [ ]:
# hyperparameter
batch_size = 64
num_workers = 3
validation_split = 0.2
num_epochs = 2

# load the dataset
train_loader, val_loader, test_loader = mnist.load_data(
    batch_size=batch_size, num_workers=num_workers, validation_split=validation_split
)

In [ ]:
# define the model
model = IBSequential(input_shape=(1, 28, 28), gpu=False, track_dynamics=True, save_dynamics=True)
model.add(Conv2d(filters=16, kernel_size=3, stride=1, padding=1))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
print(model)

## Attaching dynamics evaluator to the model


In [ ]:
# compile the model
model.compile(optimizer='SGD', loss='cross_entropy', metrics=['accuracy'])
model.attach_evaluator(HSIC(kernel='gaussian', gpu=False, sigma=5))

In [ ]:
# train the model along with calculating dynamics
model.fit_generator(train_loader, val_loader, num_epochs)